# Save all links

In [ ]:
#Win
# from selenium import webdriver

# host = "https://www.hotcar.com.tw/"
# browser = webdriver.Chrome("E:\\selenium\\chromedriver_win32\\chromedriver")
# browser.get(host)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import re
import time

# open browser
browser = webdriver.Chrome()
browser.get('http://www.yes123.com.tw/admin/index.asp')

# input search keywords
browser.find_element_by_id('find_key1').clear()
browser.find_element_by_id('find_key1').send_keys('軟體 工程師')

time.sleep(2)
# Click search button
browser.find_element_by_class_name('n_serch_btn').click()

time.sleep(2)
counter = 0

while True:
    try:
        counter += 1
        browser.find_element_by_tag_name('body').send_keys(Keys.END)    #scroll down
        links = [link.get_attribute('href') for link in browser.find_elements_by_class_name('jobname')]

        with open('link_yes123_2.csv', 'w') as f:
            f.write('\n'.join(links) + '\n')                            #twice  '\n'  這頁的links存入csv檔
        print('[Debug] links of page %d are saved' % (counter))         #Debug : which page is saved爬完一頁顯示
        browser.find_element_by_link_text('>').click()                  #find '>' in text to click
        time.sleep(1)   
    except Exception as e:                                              #Exception
        print(e)
        break
print('The end of pages is No.%d' %(counter))
print('Browser will quit!')
browser.quit()

# Get the content form one 
page


In [ ]:
def save_content(link):
    import requests 
    from bs4 import BeautifulSoup
    import re
    res = requests.get(link)
    soup = BeautifulSoup(res.text, 'lxml')
    content = ''
#     print(res)
    
    divs = soup.select('div.comp_detail')
#     print(len(divs))
    del divs[-1]
    for div in divs:
        content += div.text.replace('\n', '').replace('\xa0', '')   #**********************

    text = '/'.join(re.findall('object c|visual basic|\.net|[A-Za-z]+[+#]*', content, re.IGNORECASE))

    with open('yes123_job3.csv', 'a') as f:
        f.write(text + '\n')


# Save all contents


In [ ]:
"""讀取每行字串、每讀50筆（行）、會回報一次目前讀取的筆數
#second open link_yes123.csv to get the content"""

import time

with open('link_yes123.csv', 'r') as fr:
    link_list = fr.read().split('\n')
    
"""在link_list中,每50筆link會回報目前執行筆數.
並執行save_content方法（點擊網頁並撈出內容）,睡眠0.3毫秒;
反之,若執行遇到例外,則忽略當下該筆,繼續執行下一筆save_content方法"""    
                                        # link_list    
for idx, link in enumerate(link_list):   #use enumerate to add idx
    if idx % 50 == 0:
        print('Current link {}'.format(idx))
    try:
        save_content(link)
#         time.sleep(0.3)
    except Exception as e:               #throw in save_content function
        print(e)
        continue

# Multithreading

In [ ]:
"""執行多執行緒（Multithreading)  使多執行緒執行save_content方法、lock、並將每篇連結的內容寫入yes123_job2.csv

   註：在th.start()後 請直接跳到def save_content_th(link, lock, f) 查閱多執行緒執行run()的方法
"""

import time
import threading
#讀取'link_yes123.csv', 寫入'yes123_job2.csv'"""
with open('link_yes123.csv', 'r') as fr:
    link_list = fr.read().split('\n')
with open('yes123_job.csv', 'a') as f:
    for idx, link in enumerate(link_list):
        #每讀取50筆（行）link、回報目前讀取連結的數量
        if idx % 50 == 0:
            print('Current link {}'.format(idx))            
        try:
            lock = threading.Lock()
            """定義多執行緒Thread的工作："save_content_th",
            傳入該方法的參數args為
            （1）每筆連結
            （2）lock方法(請到save_content_th(link, lock, f)查閱詳細內容）
            （3）執行完save_content_th方法後將文章內容寫入'yes123_job2.csv'
            若執行遇到例外,則忽略該筆連結,thread繼續執行下一筆連結
            """
            th = threading.Thread(target=save_content_th, args=(link, lock, f))
            th.start()
            time.sleep(0.1)
        except Exception as e:
            print(e)
            continue
    else:
        print('Process finished')

In [ ]:
"""多執行緒的工作：save_content_th(link, lock, f)
   進入連結將內容撈出來、並針對文章內容過濾符號、正規表示法,
   並確保執行寫入f.write(text + '\n')時,只會有一個執行緒執行
"""

def save_content_th(link, lock, f):
    import requests 
    from bs4 import BeautifulSoup
    import re
    import time
    #進入連結內容、將文章的內容撈出來
    res = requests.get(link)
#     time.sleep(1)
    soup = BeautifulSoup(res.text, 'lxml')
    content = ''
    
    divs = soup.select('div.comp_detail')
#     del divs[-1]
    #將該篇文章有div標籤的內容撈出來、並去除部份符號
    for div in divs:
        content += div.text.replace('\n', '').replace('\xa0', '')
    #文章經過正規表示法、並忽略大小寫後,會變成眾多單字,用"/"連結該文章所有的單字
    text = '/'.join(re.findall('object c|visual basic|\.net|[A-Za-z]+[+#]*', content, re.IGNORECASE))
    
    #執行緒lock.acquire():從unlock轉為lock狀況,在lock.release()之前,
    #只有該執行緒thread執行f.write(text + '\n')方法,確保不會有多個執行緒同時寫入    
    
    lock.acquire()
    f.write(text + '\n')
    lock.release()

# Word Counts

In [ ]:
"""讀取過濾後用'/'串連單字的文章,並以'/'拆回各個單字,並用counter計算出現的單字數"""

# counter
from collections import Counter

counter = Counter()
with open('yes123_job.csv', 'r') as fr:
   
    for line in fr:
        
    """每行的文章長度>0,則以'/'符號、空格拆解該篇文章。
    並將該行的所有單字塞入set中,確保同樣的單字不會重複被塞入set中,再強制轉為list,準備轉成dictionary。"""
        if len(line) > 0:
            words= list(set(line.strip().split('/')))  # remove duplicate items in line
    """只要該行的單字（key)轉為小寫後,若出現過則value+1、反之value=1"""        
        for word in words:
            if word.lower() in counter.keys():
                counter[word.lower()] += 1
            else:
                counter[word.lower()] =  1


In [ ]:
"""將counter內,value排序從大到小為前50筆的單字（key)寫入'123.csv'檔案中"""
counter.most_common(50)
with open('123.csv', 'w') as f:
    for lan, counts in counter.most_common():
        f.write('{},{}\n'.format(lan,counts))